## Ntropy AI demo - Multimodal RAG on slide deck
for this demo, we'll make a RAG from the slide deck of https://static.conocophillips.com/files/2023-conocophillips-aim-presentation.pdf, that contains text, tables, images, diagrams, and complex layouts.

A gap of text-based RAG is that they struggle with purely text-based representations of complex documents. For instance, if a page contains a lot of images and diagrams, a text parser would need to rely on raw OCR to extract out text. You can also use a multimodal model (e.g. gpt-4o and up) to do text extraction, but this is inherently a lossy conversion.

the idea here is to create embeddings from slide images using a Multimodal Embeddings model, use openai tool calling to let a LLM call the retriever with the right keywords, then pass the returned images to gpt4o to let it answer the query.

this light architecture achieve satisfactory results, and further document processing before retrieval can be done.


this demo is inspired by https://github.com/run-llama/llama_parse/blob/main/examples/multimodal/multimodal_rag_slide_deck.ipynb


In [1]:
from ntropy_ai.core.auth import BaseAuth # for Ntropy Central Auth System
import os
from ntropy_ai.core.utils import clear_cache # a function to clear the cached files for the embeddings
from ntropy_ai.core.utils.base_format import Document # the Document class to store the data
from ntropy_ai.core.providers.aws import utils as aws_utils # a utility function to upload the images to S3

# hide warning
import warnings
warnings.filterwarnings('ignore')

/Users/hugolebelzic/miniconda3/envs/ntropy/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
db_instance = BaseAuth() # we first import the BaseAuth class, which is the class for the Ntropy Central Auth System
key_file = os.path.join(os.getcwd(), "private_key.pem") # we get the path to the private key file, which is the 'password' to sign in
db_instance.connect(key_file=key_file) # we connect to the database
#if everything goes well, you should see a message like this:

AWS connection initialized successfully.
OpenAI connection initialized successfully.
Pinecone connection initialized successfully.


In [3]:
from ntropy_ai.core.document_instance.load.pdf import PDFLoader
from ntropy_ai.core.document_instance.process.chunk_text import BasicTextChunk

file = '2023-conocophillips-aim-presentation.pdf'
img_path = 'pdf_images'

loaded_pdf = PDFLoader(file_path=file, output_img_path=img_path)
texts_ = loaded_pdf.extract_text()
images_ = loaded_pdf.extract_images()
chunks = []
for text in texts_:
    chunks.extend(BasicTextChunk(chunk_size=64, document=text))

In [4]:
from tqdm import tqdm # use tqdm to have a pretty loading bar

images_ = [] # initialize empty images_ list
for file in tqdm(os.listdir('pdf_images'), desc="Uploading images"): # iterate through each file
    # we're going to use the aws_utils helper function to upload the images to the s3 bucket
    # the function directly upload the image to the s3 bucket and return the url. Please check the notebook LINK for more configuration details
    img_url = aws_utils.upload_to_s3(os.path.join('pdf_images', file))
    images_.append(Document(image=img_url, metadata={'type': 'image'})) #

Uploading images: 100%|██████████| 62/62 [03:53<00:00,  3.77s/it]


In [5]:
images_

[Document(id='1543ff917f0943e7aa0d589ed6bf4232', metadata={'type': 'image'}, page_number=None, content=None, image='https://ntropy-test.s3.amazonaws.com/pdf_images/image_39_1.png'),
 Document(id='9f28419a13784e279ba4931e08d25fe1', metadata={'type': 'image'}, page_number=None, content=None, image='https://ntropy-test.s3.amazonaws.com/pdf_images/image_41_1.png'),
 Document(id='b2100388c1694e4281c524f6acec2eb2', metadata={'type': 'image'}, page_number=None, content=None, image='https://ntropy-test.s3.amazonaws.com/pdf_images/image_6_1.png'),
 Document(id='e0c8691b89594e2fbcf08fe1675b7456', metadata={'type': 'image'}, page_number=None, content=None, image='https://ntropy-test.s3.amazonaws.com/pdf_images/image_20_1.png'),
 Document(id='21a3b6187aa74813918e123991156b66', metadata={'type': 'image'}, page_number=None, content=None, image='https://ntropy-test.s3.amazonaws.com/pdf_images/image_58_1.png'),
 Document(id='15fc9ba7898b477fb882e985d3fc8a6c', metadata={'type': 'image'}, page_number=No

In [6]:
from tqdm import tqdm
from ntropy_ai.core.providers import openai # we import the openai class from the providers. a complete guide is available in the notebook LINK

embeddings = []
for doc in tqdm(images_, desc="Embedding documents"):
    # we now create the embeddings for each image, and append them to the embeddings list
    # the openai.OpenAIEmbeddings directly return us a Vector object, that contains the embedding and the original document
    embeddings.append(openai.OpenAIEmbeddings(model='openai.clip-vit-base-patch32', document=doc)) 

Embedding documents: 100%|██████████| 62/62 [01:34<00:00,  1.53s/it]


In [7]:
from ntropy_ai.core.vector_store.pinecone import Pinecone

pc = Pinecone(index_name='aws-doc') # initialize a Pinecone object
# create the index, dimension should be the same as the embeddings model. 
# metric can be chosen from the pinecone documentation
pc.create_index(index_name="slide-deck", dimension=512, metric="cosine") 
pc.set_index(index_name="slide-deck") # set the default index
pc.set_embeddings_model(model="openai.clip-vit-base-patch32", model_settings={}) # we set the default embedding model, which should be the same as the one we used to create the embeddings
# we can set a model settings but it is not required

In [8]:

from tqdm import tqdm
for v in tqdm(embeddings, desc="Adding vectors"):
    pc.add_vectors(vectors=[v]) #we add the vector individually on the pinecone vector store

Adding vectors: 100%|██████████| 62/62 [00:25<00:00,  2.43it/s]


In [34]:
from ntropy_ai.core.vector_store.pinecone import Pinecone

pc = Pinecone(index_name="slide-deck")
pc.set_index(index_name="slide-deck") # set vector store to the index we created before
pc.set_embeddings_model(model="openai.clip-vit-base-patch32", model_settings={}) # set the default embedding model, which should be the same as the one we used to create the embeddings
pc.set_retriever_settings(top_k=3, include_values=False) # we only want one results, too many image results can affect the quality of the response, especially with small models

In [36]:
# create a function to query the vector embeddings base with keywords

# we define our function
def query_db(keywords: str):
    data = pc.query(query_text=keywords)
    return ",".join([str(d.content) for d in data])

# we define the function schema according to openai tool calling format
openai_tools = [
  {
    "type": "function",
    "function": {
      "name": "query_db",
      "description": "Query the vector embeddings base with string keywords. The database is the slide deck of the Conoco Phillips AIM presentation",
      "parameters": {
        "type": "object",
        "properties": {
          "keywords": {
            "type": "string",
            "description": "The string keywords to query the vector embeddings base",
          }
        },
        "required": ["keywords"],
      },
    }
  }
]

# we define a map to link the function to the python function we defined earlier
functions_tools  = {
    "query_db": query_db
}

from ntropy_ai.core.providers import openai # we import the openai class from the providers. a complete guide is available in the notebook LINK

model = openai.OpenaiModel(
    model_name="gpt-4o", # gpt 4o on top
    tools=openai_tools, # we define the tools that we want to use
    tools_choice="required", # we define the tools choice, which is required, auto or none
    function_caller=functions_tools, # we define the function caller, which is the function to call the tools
    system_prompt='Given the query, use the tool to query the slide deck of the Conoco Phillips AIM presentation.'
)


In [37]:
query_text = "Describe the financial plan of ConocoPhilips"

In [38]:
r = model.chat(query=query_text)

[{'role': 'user', 'content': [{'type': 'text', 'text': 'Describe the financial plan of ConocoPhilips'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': ''}], 'tool_calls': [{'id': 'call_Z8DmHjOCyWoKhuqOo25rpQ3B', 'type': 'function', 'function': {'name': 'query_db', 'arguments': '{"keywords": "financial plan"}'}}]}, {'role': 'tool', 'content': [{'type': 'text', 'text': 'https://ntropy-test.s3.amazonaws.com/pdf_images/image_45_1.png,https://ntropy-test.s3.amazonaws.com/pdf_images/image_52_1.png,https://ntropy-test.s3.amazonaws.com/pdf_images/image_7_1.png'}], 'tool_call_id': 'call_Z8DmHjOCyWoKhuqOo25rpQ3B'}]


In [39]:
model.history.get_history() # we will extract the images url from the history

[{'role': 'system',
  'content': 'Given the query, use the tool to query the slide deck of the Conoco Phillips AIM presentation.',
  'images': None,
  'tool_call': None,
  'tool_call_response': None,
  'tools': None,
  'timestamp': '2024-07-17T17:52:19.312332'},
 {'role': 'user',
  'content': 'Describe the financial plan of ConocoPhilips',
  'images': None,
  'tool_call': None,
  'tool_call_response': None,
  'tools': [{'type': 'function',
    'function': {'name': 'query_db',
     'description': 'Query the vector embeddings base with string keywords. The database is the slide deck of the Conoco Phillips AIM presentation',
     'parameters': {'type': 'object',
      'properties': {'keywords': {'type': 'string',
        'description': 'The string keywords to query the vector embeddings base'}},
      'required': ['keywords']}}}],
  'timestamp': '2024-07-17T17:52:24.862128'},
 {'role': 'function',
  'content': None,
  'images': None,
  'tool_call': {'tool_name': 'query_db',
   'arguments'

In [40]:
images = model.history.get_history()[2]['tool_call_response'].split(',') # extract images url from query

# pass it to the second model
model_2 = openai.OpenaiModel(
    model_name="gpt-4o", # gpt 4o on top
    system_prompt="""Given the context information and not prior knowledge, answer the query. Explain your reasoning for the final answer"""
)
r = model_2.chat(
    query=query_text,
    images=images
)

In [41]:
print(r) # final results

The financial plan of ConocoPhillips, as outlined in the provided images, is a comprehensive 10-year strategy spanning from 2023 to 2032. Key elements of the plan include:

### Sources of Funds:
- CFO at $60/BBL (Barrel) WTI Mid-Cycle Planning Price: Approximately $200 billion.
- CFO at $80/BBL WTI Upside Sensitivity: Increased funds reaching up to approximately $300 billion.
- Cash includes cash, cash equivalents, restricted cash, and short-term investments.

### Uses of Funds:
- **Capital Expenditures**
- **30% of CFO Distribution Commitment:** Ensures a significant portion of cash flow from operations is committed to distributions.
- **Additional Distributions:** Extra funds allocated for distributions over the planned commitment.

### Financial Goals and Metrics:
- **Return on Capital Employed (ROCE):** Aim for peer-leading ROCE improvement over time.
- **Dividend Growth:** Target top quartile ordinary dividend growth.
- **Market Cap Distribution:** Plans to distribute over 90% of 

The financial plan of ConocoPhillips, as outlined in the provided images, is a comprehensive 10-year strategy spanning from 2023 to 2032. Key elements of the plan include:

### Sources of Funds:
- CFO at $60/BBL (Barrel) WTI Mid-Cycle Planning Price: Approximately $200 billion.
- CFO at $80/BBL WTI Upside Sensitivity: Increased funds reaching up to approximately $300 billion.
- Cash includes cash, cash equivalents, restricted cash, and short-term investments.

### Uses of Funds:
- **Capital Expenditures**
- **30% of CFO Distribution Commitment:** Ensures a significant portion of cash flow from operations is committed to distributions.
- **Additional Distributions:** Extra funds allocated for distributions over the planned commitment.

### Financial Goals and Metrics:
- **Return on Capital Employed (ROCE):** Aim for peer-leading ROCE improvement over time.
- **Dividend Growth:** Target top quartile ordinary dividend growth.
- **Market Cap Distribution:** Plans to distribute over 90% of the market capitalization, which is based on a market cap of approximately $121 billion as of March 31, 2023.
- **WTI FCF Breakeven:** Targeting a breakeven price of approximately $35 per barrel of WTI.
- **CFO and FCF CAGR:** Financial plans aim for a Compound Annual Growth Rate (CAGR) of about 6% for cash flow from operations (CFO) and around 11% for free cash flow (FCF).

### Additional Notes:
- The plan does not hedge for price upside, allowing it to potentially benefit more directly from favorable market conditions.
- These financial objectives emphasize strong capital returns, sustainable growth, and substantial shareholder distributions.

This robust financial strategy demonstrates ConocoPhillips' commitment to optimizing financial performance, enhancing shareholder value, and maintaining capital discipline over the next decade.
